In [ ]:
# | hide
from fast_kafka_api.core import *

# fast-kafka-api

> Extension of FastAPI with Kafka event handlers

This file will become your README and also the index of your documentation.

## Install

```sh
pip install fast_kafka_api
```

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
1 + 1

2